In [310]:
# !pip install faker
# !pip install babel
# !pip install tensorflow
# !pip install numpy
# !pip install matplotlib
# !pip install tqdm


In [311]:
# !pip install wandb

In [312]:
!wandb login

wandb: Currently logged in as: ducanh2002add (ducanh2002add-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [313]:
import utils

In [314]:
total_examples = 20000
dataset, human_vocab, machine_vocab, inv_machine_vocab = utils.load_dataset(total_examples)
# Train parameters
num_epochs = 100
batch_size = total_examples
input_dim = len(human_vocab)
output_dim = len(machine_vocab)
hidden_dim = 512
learning_rate = 0.00004
batch_size = 512
num_layers = 5

100%|██████████| 20000/20000 [00:00<00:00, 36046.93it/s]


In [315]:
import wandb
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="ducanh2002add-hanoi-university-of-science-and-technology",
    # Set the wandb project where this run will be logged.
    project="date-translation-project",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": learning_rate,
        "architecture": "LSTM",
        "dataset": "faker + babel",
        "epochs": num_epochs,
        "num_layers": num_layers,
        "hidden_dim": hidden_dim,
        "batch_size": batch_size,
        "total_examples": total_examples,
        "input_dim": input_dim,
        "output_dim": output_dim,

    }
)

Loss,█▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss,1.21053


In [316]:
dataset[:5]

[('tuesday january 17 2006', '=2006-01-17'),
 ('14 august 1982', '=1982-08-14'),
 ('friday november 1 1991', '=1991-11-01'),
 ('21 may 2007', '=2007-05-21'),
 ('20 sep 2015', '=2015-09-20')]

In [317]:
maximum_human_length = 30
maximum_machine_length = 11
X, Y, X_onehot, Y_onehot = utils.preprocess_data(dataset, human_vocab,
                                                 machine_vocab,
                                                 maximum_human_length,
                                                 maximum_machine_length)

In [318]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36,
 '<eos>': 37,
 '=': 38}

In [319]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '=': 11,
 '<unk>': 12,
 '<pad>': 13,
 '<eos>': 14}

In [320]:
print("X.shape: ", X.shape)
print("Y.shape: ", Y.shape)
print("X_onehot.shape", X_onehot.shape)
print("Y_onehot.shape: ", Y_onehot.shape)

X.shape:  (20000, 30)
Y.shape:  (20000, 11)
X_onehot.shape (20000, 30, 39)
Y_onehot.shape:  (20000, 11, 15)


In [321]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", X_onehot[index])
print("Target after preprocessing (one-hot):", Y_onehot[index])

Source date: tuesday january 17 2006
Target date: =2006-01-17

Source after preprocessing (indices): [30 31 17 29 16 13 34  0 22 13 25 31 13 28 34  0  4 10  0  5  3  3  9 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [11  3  1  1  7  0  1  2  0  2  8]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Target after preprocessing (one-hot): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 

In [322]:
import torch
print(torch.cuda.is_available())
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
print(torch.__version__)

True
cuda
2.6.0+cu124


In [323]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, dropout = 0.):
    super().__init__()
    self.device = device
    self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout)

  # src is onehot_encoding
  def forward(self, src):
    outputs, (hidden, cell) = self.rnn(src)
    return hidden, cell

In [324]:
encoder_test = Encoder(len(human_vocab), hidden_dim, 1).to(device)
input_test = torch.tensor(data = X_onehot, dtype = torch.float, device = device).swapaxes(0, 1)
print("input_test.shape: (step_size, batch_size, len(human_vocab)) \n", input_test.shape)
print(encoder_test.device)
hidden_test, cell_test = encoder_test(input_test)
print("hidden_test.shape: (step_size, batch_size, hidden_size) \n", hidden_test.shape)
print("cell_test.shape: (step_size, batch_size, hidden_size) \n", cell_test.shape)

input_test.shape: (step_size, batch_size, len(human_vocab)) 
 torch.Size([30, 20000, 39])
cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.72 GiB. GPU 0 has a total capacity of 14.74 GiB of which 810.12 MiB is free. Process 73837 has 13.95 GiB memory in use. Of the allocated memory 12.34 GiB is allocated by PyTorch, and 1.47 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
class Decoder(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super(Decoder, self).__init__()
    self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, X, hidden, cell):
    # Shape of X: (batch_size, len(machine_vocab)), X is onehot encoding
    X = X.unsqueeze(0)
    output, (hidden, cell) = self.rnn(X, (hidden, cell))
    predictions = self.fc(output)
    predictions = predictions.squeeze(0)
    return predictions, hidden, cell

In [ ]:
decoder_test = Decoder(len(machine_vocab), hidden_dim, len(machine_vocab), 1).to(device)
input_test = torch.tensor(data = Y_onehot[:, 0, :], dtype = torch.float, device = device)
print("input_test.shape: (batch_size, len(machine_vocab)) \n", input_test.shape)
prediction_test, hidden_test, cell_test = decoder_test(input_test, hidden_test, cell_test)
print("prediction_test.shape: (batch_size, len(machine_vocab))\n", prediction_test.shape)
print("hidden_test.shape: (1 = step_size, batch_size, hidden_size)\n", hidden_test.shape)
print("cell_test.shape: (1 = step_size, batch_size, hidden_size)\n", cell_test.shape)

In [ ]:
import numpy
class Seq2Seq(nn.Module):
  def __init__(self, encoder : Encoder, decoder : Decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
  def get_tensor(self, src, tgt):
    src = src.swapaxes(0, 1)
    if type(src) == numpy.ndarray:
      src = torch.tensor(data = src, dtype = torch.float, device = device)
    if type(tgt) == numpy.ndarray:
      tgt = torch.tensor(data = tgt, dtype = torch.float, device = device)
    # src = torch.tensor(data = src, dtype = torch.float, device = device).swapaxes(0, 1)
    # tgt = torch.tensor(data = tgt, dtype = torch.float, device = device)
    return src, tgt
  def forward(self, src, tgt, teacher_forcing_ratio = 0.5):
    src, tgt = self.get_tensor(src, tgt)

    batch_size = src.shape[1]
    trg_len = tgt.shape[1]
    trg_vocab_size = len(machine_vocab)

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

    hidden, cell = self.encoder(src)

    x = tgt[:, 0, :]
    for t in range(1, trg_len):
      x, hidden, cell = self.decoder(x, hidden, cell)

      outputs[t] = x
      # shape: (batch_size, 1)
      best_guess = x.argmax(dim = 1)

      x = nn.functional.one_hot(x.argmax(dim = 1), num_classes = trg_vocab_size).to(torch.float)

    return outputs.swapaxes(0, 1)

In [ ]:
model_test = Seq2Seq(encoder_test, decoder_test).to(device)
outputs = model_test(X_onehot, Y_onehot)
print("outputs.shape: (batch_size, trg_len - 1, trg_vocab_size)\n", outputs.shape)

In [ ]:
machine_vocab['<pad>']

In [ ]:
encoder = Encoder(len(human_vocab), hidden_dim, num_layers).to(device)
decoder = Decoder(len(machine_vocab), hidden_dim, len(machine_vocab), num_layers).to(device)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index = machine_vocab['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

X_onehot_tensor = torch.tensor(data = X_onehot, dtype = torch.float, device = device)
Y_onehot_tensor = torch.tensor(data = Y_onehot, dtype = torch.float, device = device)
Y_tensor = torch.tensor(data = Y[:, :], device = device)

# Tạo DataLoader
dataset1 = TensorDataset(X_onehot_tensor, Y_onehot_tensor, Y_tensor)
dataloader = DataLoader(dataset1, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (X_batch, Y_onehot_batch, Y_batch) in enumerate(dataloader):
    # X_batch, Y_onehot_batch, Y_batch = batch
    y_pred = model(X_batch, Y_onehot_batch)

    y_pred = y_pred.reshape(-1, y_pred.shape[-1])
    Y_batch = Y_batch.reshape(-1)

    l = loss_fn(y_pred, Y_batch)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

    print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {l.item()}")
    run.log(
        {
            "Loss": l.item()
        }
    )

In [ ]:
# X_onehot_tensor = torch.tensor(data = X_onehot, dtype = torch.float, device = device)
# Y_onehot_tensor = torch.tensor(data = Y_onehot, dtype = torch.float, device = device)
# Y_tensor = torch.tensor(data = Y[:, :], device = device)
# for epoch in range(num_epochs):
#   model.train()

#   y_pred = model(X_onehot_tensor, Y_onehot_tensor)

#   y_pred = y_pred.reshape(-1, y_pred.shape[-1])  # Reshape to [batch_size * target_sequence_length, target_vocabulary_size]
#   Y_tensor = Y_tensor.reshape(-1)
#   # print("y_pred.shape: ", y_pred.shape)
#   # print("Y_tensor.shape: ", Y_tensor.shape)
#   l = loss_fn(y_pred, Y_tensor)
#   optimizer.zero_grad()
#   l.backward()
#   optimizer.step()
#   print("Epoch: ", epoch, "Loss: ", l.item())


In [ ]:
import numpy as np

In [ ]:
total_examples = 10
dataset, _, _, _ = utils.load_dataset(total_examples)

In [ ]:
EXAMPLES = []
for i in range(min(10, len(dataset))):
  EXAMPLES.append(dataset[i][0])

In [ ]:
EXAMPLES

In [ ]:
def predict(model, src_sentence, num_preds = 11):
    list_tokens = utils.string_to_int(src_sentence, 30, human_vocab)
    # print(list_tokens)
    X = torch.tensor(list_tokens, device = device)
    X = nn.functional.one_hot(X, num_classes = len(human_vocab)).to(torch.float)
    # print(list_tokens)
    # print(X)
    X = X.unsqueeze(0).swapaxes(0, 1)
    # print(X.shape)
    with torch.no_grad():
        hidden, cell = model.encoder(X)
    X = torch.tensor(machine_vocab['='], device = device).unsqueeze(0)
    X = nn.functional.one_hot(X, num_classes = len(machine_vocab)).to(torch.float)
    # print(X.shape)

    result = []
    for _ in range(num_preds):
        with torch.no_grad():
            X, hidden, cell = model.decoder(X, hidden, cell)
            # X = X.argmax(1)

            best_guess = X.argmax(dim = 1)

            X = nn.functional.one_hot(X.argmax(dim = 1), num_classes = len(machine_vocab)).to(torch.float)
        result.append(best_guess.cpu().numpy()[0])

    # print(data.tgt_vocab.to_tokens(result))
    final_sentene = [inv_machine_vocab[x] for x in result]
    return "".join(final_sentene[:-1])
for txt in EXAMPLES:
  print(f"{txt}  ->  {predict(model, txt)} ")